In [81]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [82]:
dataframe = pd.read_excel("Gem comparison products copy.xlsx")
dataframe = dataframe.loc[:, ['URL', 'CURRENT PRICE']]
p_index = 2

In [83]:
dataframe.head()

,URL,CURRENT PRICE
0,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,0
1,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,0
2,http://mkp.gem.gov.in/mcb-miniature-circuit-br...,0
3,https://mkp.gem.gov.in/mcb-miniature-circuit-b...,0


In [84]:
product_url = dataframe["URL"][p_index]
print(product_url)

http://mkp.gem.gov.in/mcb-miniature-circuit-breakers-c-operation-per-iec/16a-fp-c-mcb/p-5116877-76167892508-cat.html


In [85]:
product_url = "https://mkp.gem.gov.in/mcb-miniature-circuit-breakers-c-operation-per-iec/3a-tpn-c-mcb/p-5116877-6726934656-cat.html#variant_id=5116877-6726934656"

In [86]:
# product_url = "https://mkp.gem.gov.in/mcb-miniature-circuit-breakers-c-operation-per-iec/20a-sp-c-mcb/p-5116877-295066335-cat.html#variant_id=5116877-295066335"

In [87]:
competitors_url = re.sub(".html.*", "/all_sellers.html", product_url)

In [88]:
competitors_url_response = requests.get(competitors_url)

In [89]:
competitors_url_soup = BeautifulSoup(competitors_url_response.content, 'html.parser')

In [90]:
competitors_prices_table = competitors_url_soup.find('div', attrs={'id': 'sellers-table-wrap'})
competitors_prices_table = competitors_prices_table.find('table')
for div in competitors_prices_table.find_all("div", {'class':'seller-info-table'}):
    div.decompose()

In [91]:
table_rows = competitors_prices_table.find_all('tr')

res = []
for i, tr in enumerate(table_rows):
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["Sellers", "Offer Price", "Delivery Locations", "Quantity based Discount", "Quantity Available", "Min Quantity / Consignee", "Offer Product As", "Country of Origin"])
df = df[["Offer Price", "Delivery Locations", "Quantity based Discount", "Min Quantity / Consignee"]]

In [92]:
df

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee
0,"₹2,349.99",All India,,15
1,"₹2,358.00","State(s) - WEST BENGAL, BIHAR, SIKKIM, ODISHA,...",,1


In [93]:
delivery_locations_regex = "|".join(["TAMIL NADU", "All India"])
competitors_details = df[
    df["Delivery Locations"].str.contains(delivery_locations_regex)
]

In [94]:
competitors_details

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee
0,"₹2,349.99",All India,,15


In [95]:
competitors_details["Offer Price"] = (
    competitors_details["Offer Price"].str.slice(1)
)
competitors_details["Offer Price"] = (
    competitors_details["Offer Price"].str.replace(",", "").astype(float)
)

C:\Users\nikun\AppData\Local\Temp\ipykernel_10276\717331536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitors_details["Offer Price"] = (
C:\Users\nikun\AppData\Local\Temp\ipykernel_10276\717331536.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitors_details["Offer Price"] = (


In [96]:
def determine_final_price(row):
    discount_info = row["Quantity based Discount"]

    if discount_info == "":
        return row["Offer Price"]

    discounts = discount_info.split("\n")
    final_price = row["Offer Price"]

    for discount_entry in discounts:
        discount_entry = discount_entry.strip()
        parts = discount_entry.split(" ")
        discount = float(parts[5][1:].replace(",", ""))
        price = row["Offer Price"] - discount
        min_quantity = int(parts[0])

        if price * min_quantity <= 25000 and price < final_price:
            final_price = price

    return final_price

In [97]:
competitors_details["Final Price"] = competitors_details.apply(
    determine_final_price, axis=1
)

C:\Users\nikun\AppData\Local\Temp\ipykernel_10276\3674593290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  competitors_details["Final Price"] = competitors_details.apply(


In [98]:
competitors_details

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee,Final Price
0,2349.99,All India,,15,2349.99


In [99]:
# Drop rows where Min Quantity * Offer Price exceeds 25000
competitors_details = competitors_details[
    competitors_details["Min Quantity / Consignee"].astype(int) * competitors_details["Offer Price"]
    <= 25000
]

In [100]:
competitors_details

,Offer Price,Delivery Locations,Quantity based Discount,Min Quantity / Consignee,Final Price


In [101]:
# for index, row in filtered_df.iterrows():
#     if row["Quantity based Discount"] != "":
#         discount_string = row["Quantity based Discount"].split(" ")
#         discount = float("".join(discount_string[5][1:].split(",")))
#         # min_quantity = int(discount_string[0])
#         # max_quantity = int(discount_string[2])
#         new_offer_price = row["Offer Price"] - discount
#         # temp = 25000/new_offer_price
#         # print("Values:", row["Offer Price"], "|", discount, "|", new_offer_price, "|", temp, "|", min_quantity, "|", max_quantity)
#         # if temp >= min_quantity and temp <= max_quantity:
#         # if new_offer_price * row["Min Quantity / Consignee"]
#         row["Offer Price"] = new_offer_price
#         print("Price changed:", index)
#     if int(row["Min Quantity / Consignee"]) * row["Offer Price"] > 25000:
#         filtered_df.drop(index)
#         print("Dropped:", index)

In [102]:
competitors_min_price = competitors_details['Final Price'].min()

In [103]:
if competitors_min_price == "":
    competitors_min_price = "Put your price"
print(f"Minimum price: {competitors_min_price}")

Minimum price: nan


float

In [104]:
competitors_min_price

nan

In [105]:
product_price = dataframe["CURRENT PRICE"][p_index]

In [106]:
product_price

0

In [107]:
product_price > competitors_min_price

False